# **Clone**

In [ ]:
%cd /content
!git clone https://github.com/harry2141985/faceoff.git
%cd faceoff

# **Requirements**

In [ ]:
!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

# **FFMPEG**

In [ ]:
!apt-get update
!apt-get install -y libglib2.0-0 libgl1
!apt-get update
!apt-get install -y ffmpeg

# **Models**

In [ ]:
!wget -O /content/faceoff/inswapper_128.onnx \
https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx

In [ ]:
!# Create folder
!mkdir -p models/buffalo_l

# Download buffalo_l.zip
!wget -O models/buffalo_l/buffalo_l.zip \
https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip

# Extract zip into the folder
!unzip -o models/buffalo_l/buffalo_l.zip -d models/buffalo_l

# Remove the zip file to keep directory clean
!rm models/buffalo_l/buffalo_l.zip

In [ ]:
import os
import gdown

# Define folder and file paths
model_folder = "/content/faceoff/models/gfpgan"
model_path = os.path.join(model_folder, "GFPGANv1.4.pth")

# Create folder if it doesn't exist
os.makedirs(model_folder, exist_ok=True)

# Download GFPGANv1.4.pth from Hugging Face
url = "https://huggingface.co/gmk123/GFPGAN/resolve/main/GFPGANv1.4.pth"
gdown.download(url, model_path, quiet=False)

print(f"Model downloaded to: {model_path}")

# **Fixing Error**

In [ ]:
import os
import importlib.util

# Dynamically find the basicsr package location
try:
    basicsr_spec = importlib.util.find_spec("basicsr")
    if basicsr_spec is None or basicsr_spec.origin is None:
        print("❌ basicsr package not found. Make sure it's installed via requirements.txt")
    else:
        # Get the package directory
        basicsr_path = os.path.dirname(basicsr_spec.origin)
        file_path = os.path.join(basicsr_path, "data", "degradations.py")
        
        if not os.path.exists(file_path):
            print(f"❌ File not found at: {file_path}")
        else:
            # Read the file
            with open(file_path, "r") as f:
                content = f.read()
            
            # Replace the incorrect import with the correct one
            patched = content.replace(
                "from torchvision.transforms.functional_tensor import rgb_to_grayscale",
                "from torchvision.transforms.functional import rgb_to_grayscale"
            )
            
            # Write the patched content back
            with open(file_path, "w") as f:
                f.write(patched)
            
            print("✅ Patch applied successfully!")
            print(f"📁 Patched file at: {file_path}")
            print("Updated import:")
            print("from torchvision.transforms.functional import rgb_to_grayscale")
except Exception as e:
    print(f"❌ Error applying patch: {e}")
    print("You may need to apply this fix manually or skip if the error doesn't occur.")

# **Main Code**

In [ ]:
!python main.py